In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from statsmodels.tsa.ar_model import AutoReg

In [14]:
df = pd.read_csv('us_retail_sales.csv')
X=df.values
print('Shape of data \t',df.shape)
print('\n')
print('Original Dataset:\n',df.head())
print('\n')


Shape of data 	 (30, 13)


Original Dataset:
    YEAR     JAN     FEB     MAR     APR     MAY     JUN       JUL       AUG  \
0  1992  146925  147223  146805  148032  149010  149800  150761.0  151067.0   
1  1993  157555  156266  154752  158979  160605  160127  162816.0  162506.0   
2  1994  167518  169649  172766  173106  172329  174241  174781.0  177295.0   
3  1995  182413  179488  181013  181686  183536  186081  185431.0  186806.0   
4  1996  189135  192266  194029  194744  196205  196136  196187.0  196218.0   

        SEP       OCT       NOV       DEC  
0  152588.0  153521.0  153583.0  155614.0  
1  163258.0  164685.0  166594.0  168161.0  
2  178787.0  180561.0  180703.0  181524.0  
3  187366.0  186565.0  189055.0  190774.0  
4  198859.0  200509.0  200174.0  201284.0  




In [24]:
df2 = pd.melt(df, id_vars=['YEAR'], var_name="MONTH", value_name="SALES")
df2['Month'] = df2['MONTH'].map({'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4,'MAY':5,'JUN':6,'JUL':7,'AUG':8,'SEP':9,'OCT':10,'NOV':11,'DEC':12})

In [30]:
df2['DATE']=pd.to_datetime(df2[['YEAR','Month']].assign(DAY=1))

In [31]:
df2

,YEAR,MONTH,SALES,Month,dateInt,DATE
0,1992,JAN,146925.0,1,199201,1992-01-01
1,1993,JAN,157555.0,1,199301,1993-01-01
2,1994,JAN,167518.0,1,199401,1994-01-01
3,1995,JAN,182413.0,1,199501,1995-01-01
4,1996,JAN,189135.0,1,199601,1996-01-01
...,...,...,...,...,...,...
355,2017,DEC,433282.0,12,201712,2017-12-01
356,2018,DEC,434803.0,12,201812,2018-12-01
357,2019,DEC,458055.0,12,201912,2019-12-01
358,2020,DEC,484782.0,12,202012,2020-12-01


In [32]:
df_clean = df2[['DATE','SALES']]

In [36]:
df_clean.sort_values('DATE')

,DATE,SALES
0,1992-01-01,146925.0
30,1992-02-01,147223.0
60,1992-03-01,146805.0
90,1992-04-01,148032.0
120,1992-05-01,149010.0
...,...,...
239,2021-08-01,NaN
269,2021-09-01,NaN
299,2021-10-01,NaN
329,2021-11-01,NaN
